<center><img src='https://www.datahack.es/wp-content/uploads/2020/02/logo-home.png' width="500"></center>

# Ejercicio recomendador tipo ranking 1 PUNTO
En este ejercicio os propongo generar un ranking similar al que hemos visto en el apartado de algoritmos tipo ranking. En este caso en concreto lo vamos a generar sobre los datos reales de IMDb.

Estos datos se pueden descargar de :

https://www.imdb.com/interfaces/

Al ser una base de datos muy pesada, he descargado y filtrado los datos de las películas que tengan al menos 25.000 votos.

**si para el ejercicio necesitas info extra no dudes en descargarla de la url anterior**

<center><img src='https://static.amazon.jobs/teams/53/images/IMDb_Header_Page.jpg?1501027252' width="800"></center>

## Información


Se han creado dos ficheros csv:

- **ratings_imdb.csv:** Este fichero contiene el identificador único de IMDb, el rating y el número de votos.

- **titulos_imdb.csv:** Este fichero contiene junto con el  identificador único de IMDb, información que puede resultar de interés para realizar el ejercicio.

## Pasos a seguir

1. Descarga la información de IMDb en una carpeta que se llame `datos_ejercicios/imdb` .

2. Lee los ficheros csv.

3. Crea la función que nos permite calcular la media de IMDb.

4. Prepara los datos que quieras mostrar para cada uno de los títulos.

5. Extrae el top 25 del ranking. Se valora la creatividad. 

6. Descarga el ipynb, conviertelo a html y esos serán tus documentos de evaluación.

##0. Instalaciones y librerías

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Instalación de Spark, Findspark y Pyspark.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 204.2MB 57kB/s 
     |████████████████████████████████| 204kB 46.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=200ebfe766caf415be9f92648a666492ff1a82e1b16736d2f57059bcf31ea09d
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


Creación de la Sesión Spark "ejercicio_recomendadores_tipo_ranking_rehecho".

In [ ]:
# Set up required environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ejercicio_recomendadores_tipo_ranking").master("local[*]").getOrCreate()

Instalación y carga de librerías:

In [ ]:
#Carga de datos de la web:
!pip install wget 
import wget

#Tratamiento de datos:
import pandas as pd
import numpy as np
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#Dibujo de tablas y gráficos:
pd.set_option('display.max_rows', 50) #max 50 rows out

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=d77464495325e1772238e3d919a76a1ed459e0a1be58c239fc1b1176a1dcba5a
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


##1. Descarga la información de IMDb en una carpeta que se llame datos_ejercicios/imdb.

In [ ]:
'''En lugar de descargar de la web, he copiado con el código los archivos que tenía en la otra ruta, en la nueva'''
#!cp '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/imdb/ratings_imdb.csv' '/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark'
#!cp '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/imdb/titulos_imdb.csv' '/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark'

In [ ]:
'''La descarga de la web sería'''
wget.download('https://datasets.imdbws.com/title.crew.tsv.gz', '/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark')
#wger.download('https://datasets.imdbws.com/name.basics.tsv.gz', '/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark')

'/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark/title.crew.tsv.gz'

In [ ]:
id_directores=pd.read_csv("/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark/title.crew.tsv.gz",sep="\t",compression="gzip")

In [ ]:
id_directores

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
7262550,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7262551,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7262552,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7262553,tt9916856,nm10538645,nm6951431


In [ ]:
id_directores_Spark= spark.read.csv(
    
    "/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark/title.crew.tsv.gz",
    quote='"',header =True,inferSchema=True,sep='\t')


In [ ]:
id_directores_Spark.limit(5).show(truncate=False)

+---------+---------+-------+
|tconst   |directors|writers|
+---------+---------+-------+
|tt0000001|nm0005690|\N     |
|tt0000002|nm0721526|\N     |
|tt0000003|nm0721526|\N     |
|tt0000004|nm0721526|\N     |
|tt0000005|nm0005690|\N     |
+---------+---------+-------+



##2. Lee los ficheros csv.

In [ ]:
#Voy a leer los que nos dio el profesor que son los del ejercicio
#Primero leo y retoco títulos
titulos= spark.read.csv('/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark/Ranking_IMDB/titulos_imdb.csv',
                        inferSchema=True, header=True, sep=',', quote='"').cache()

In [ ]:
titulos.limit(5).show()

+---------+-----+--------------------+--------------------+---------+-----+---------------+--------------------+--------------------+
| title_id| type|       primary_title|      original_title|premiered|ended|runtime_minutes|              titulo|              genres|
+---------+-----+--------------------+--------------------+---------+-----+---------------+--------------------+--------------------+
|tt0010323|movie|The Cabinet of Dr...|Das Cabinet des D...|     1920| null|             76|                null|Fantasy,Horror,My...|
|tt0012349|movie|             The Kid|             The Kid|     1921| null|             68|            El chico| Comedy,Drama,Family|
|tt0013442|movie|           Nosferatu|Nosferatu, eine S...|     1922| null|             94|           Nosferatu|      Fantasy,Horror|
|tt0015324|movie|        Sherlock Jr.|        Sherlock Jr.|     1924| null|             45|                null|Action,Comedy,Rom...|
|tt0015648|movie| Battleship Potemkin|Bronenosets Potemkin|   

In [ ]:
titulos.printSchema()

root
 |-- title_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- primary_title: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- premiered: integer (nullable = true)
 |-- ended: string (nullable = true)
 |-- runtime_minutes: integer (nullable = true)
 |-- titulo: string (nullable = true)
 |-- genres: string (nullable = true)



In [ ]:
titulos.count()

4974

In [ ]:
#Comprobamos que no tengamos más nulos
from pyspark.sql.functions import isnull, when, count, col

titulos.select([count(when(isnull(c), c)).alias(c) for c in titulos.columns]).show()

+--------+----+-------------+--------------+---------+-----+---------------+------+------+
|title_id|type|primary_title|original_title|premiered|ended|runtime_minutes|titulo|genres|
+--------+----+-------------+--------------+---------+-----+---------------+------+------+
|       0|   0|            0|             0|        0| 4974|              0|  1041|     0|
+--------+----+-------------+--------------+---------+-----+---------------+------+------+



In [ ]:
#Todos son nulos en la columna 'ended' y en 'titulo' hay algunos.
#Cambiamos el nombre de la columna titulo y rellenamos los nulos con ¿?. Nos cargamos la columna ended
titulos= (
    titulos
    .fillna('¿?',subset='titulo')
    .withColumnRenamed('titulo', 'titulo_espanol')
    .drop('ended')
)

titulos.limit(5).show(truncate=False)

+---------+-----+---------------------------+-------------------------------------+---------+---------------+---------------------+----------------------+
|title_id |type |primary_title              |original_title                       |premiered|runtime_minutes|titulo_espanol       |genres                |
+---------+-----+---------------------------+-------------------------------------+---------+---------------+---------------------+----------------------+
|tt0010323|movie|The Cabinet of Dr. Caligari|Das Cabinet des Dr. Caligari         |1920     |76             |¿?                   |Fantasy,Horror,Mystery|
|tt0012349|movie|The Kid                    |The Kid                              |1921     |68             |El chico             |Comedy,Drama,Family   |
|tt0013442|movie|Nosferatu                  |Nosferatu, eine Symphonie des Grauens|1922     |94             |Nosferatu            |Fantasy,Horror        |
|tt0015324|movie|Sherlock Jr.               |Sherlock Jr.             

In [ ]:
titulos.distinct().count()

4974

In [ ]:
#Volvemos a comprobar los nulos y si se ha eliminado la columna
titulos.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in titulos.columns]).show()

+--------+-------------+--------------+---------+-----+---------------+------+------+
|title_id|primary_title|original_title|premiered|ended|runtime_minutes|titulo|genres|
+--------+-------------+--------------+---------+-----+---------------+------+------+
|       0|            0|             0|        0| 4974|              0|  1041|     0|
+--------+-------------+--------------+---------+-----+---------------+------+------+



In [ ]:
#Vemos si el type sólo es movie
print('Número de type=movie:',titulos.filter(F.col('type') == 'movie').count())
print('Número de datos totales:',titulos.count())

Número de type=movie: 4974
Número de datos totales: 4974


In [ ]:
#Son todos iguales, luego nos lo cargamos
titulos= titulos.drop('type')

titulos.show(truncate=False)

+---------+------------------------------+-------------------------------------+---------+-----+---------------+---------------------------+-----------------------+
|title_id |primary_title                 |original_title                       |premiered|ended|runtime_minutes|titulo                     |genres                 |
+---------+------------------------------+-------------------------------------+---------+-----+---------------+---------------------------+-----------------------+
|tt0010323|The Cabinet of Dr. Caligari   |Das Cabinet des Dr. Caligari         |1920     |null |76             |null                       |Fantasy,Horror,Mystery |
|tt0012349|The Kid                       |The Kid                              |1921     |null |68             |El chico                   |Comedy,Drama,Family    |
|tt0013442|Nosferatu                     |Nosferatu, eine Symphonie des Grauens|1922     |null |94             |Nosferatu                  |Fantasy,Horror         |
|tt0015324

In [ ]:
#Vemos duplicados
print(titulos.select(F.countDistinct('title_id')).show())

+------------------------+
|count(DISTINCT title_id)|
+------------------------+
|                    4937|
+------------------------+

None


In [ ]:
#Hay duplicados, los eliminaremos después de unir las tablas

In [ ]:
#Leemos y retocamos el otro dataset
ratings= spark.read.csv('/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark/Ranking_IMDB/ratings_imdb.csv',
                        inferSchema=True, quote='"',sep=',',header=True).cache()

In [ ]:
ratings.show()

+---------+------+------+
| title_id|rating| votes|
+---------+------+------+
|tt0010323|   8.1| 51891|
|tt0012349|   8.3|102198|
|tt0013442|   7.9| 83163|
|tt0015324|   8.2| 35443|
|tt0015648|   8.0| 49394|
|tt0015864|   8.2| 92335|
|tt0017136|   8.3|148997|
|tt0017925|   8.1| 74070|
|tt0018455|   8.1| 44438|
|tt0019254|   8.2| 42456|
|tt0020629|   8.0| 54473|
|tt0021749|   8.5|153361|
|tt0021814|   7.5| 42833|
|tt0021884|   7.8| 60562|
|tt0022100|   8.3|132541|
|tt0022913|   7.9| 39519|
|tt0023969|   7.8| 53199|
|tt0024184|   7.7| 27265|
|tt0024216|   7.9| 75249|
|tt0025316|   8.1| 87032|
+---------+------+------+
only showing top 20 rows



In [ ]:
ratings.printSchema()

root
 |-- title_id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- votes: integer (nullable = true)



In [ ]:
#Vemos si hay nulos
ratings.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in ratings.columns]).show()

+--------+------+-----+
|title_id|rating|votes|
+--------+------+-----+
|       0|     0|    0|
+--------+------+-----+



In [ ]:
#Vemos que tenemos el mismo número de datos en ratings y en titulos
print('Número de datos ratings: ',ratings.count())
print('Número de datos titulos:',titulos.count())

Número de datos ratings:  4974
Número de datos titulos: 4974


In [ ]:
#Vemos duplicados
print(ratings.select(F.countDistinct('title_id')).show())


+------------------------+
|count(DISTINCT title_id)|
+------------------------+
|                    4937|
+------------------------+

None


In [ ]:
#Tenemos duplicados, los eliminaremos despues de unir las tablas

In [ ]:
#Unimos los dos dataset por title_id

peliculas= titulos.join(ratings, on='title_id').cache()

peliculas.show()

+---------+--------------------+--------------------+---------+---------------+--------------------+--------------------+------+------+
| title_id|       primary_title|      original_title|premiered|runtime_minutes|      titulo_espanol|              genres|rating| votes|
+---------+--------------------+--------------------+---------+---------------+--------------------+--------------------+------+------+
|tt0022100|                   M|M - Eine Stadt su...|     1931|            117|M, el vampiro de ...|Crime,Mystery,Thr...|   8.3|132541|
|tt0093756|Police Academy 4:...|Police Academy 4:...|     1987|             88|Loca academia de ...|        Comedy,Crime|   5.0| 38032|
|tt0094963|       The Dead Pool|       The Dead Pool|     1988|             91|      La lista negra|Action,Crime,Mystery|   6.3| 37906|
|tt0110367|        Little Women|        Little Women|     1994|            115|          Mujercitas|Drama,Family,Romance|   7.3| 45270|
|tt0227445|           The Score|           The S

In [ ]:
#Comprobamos de nuevo que no haya nulos, pa porsi
peliculas.select([count(when(isnull(c), c)).alias(c) for c in peliculas.columns]).show()

+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+
|title_id|primary_title|original_title|premiered|runtime_minutes|titulo_espanol|genres|rating|votes|
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+
|       0|            0|             0|        0|              0|             0|     0|     0|    0|
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+



In [ ]:
#Comprobamos duplicados
print('Total ids diferentes:')
print(peliculas.select(F.countDistinct('title_id')).show())
print('Total datos:')
print(peliculas.count())

Total ids diferentes:
+------------------------+
|count(DISTINCT title_id)|
+------------------------+
|                    4937|
+------------------------+

None
Total datos:
5050


In [ ]:
#OJOOO!!! Algo pasa... ¿Tenemos duplicados?
print('Distintos')
print(peliculas.select([F.countDistinct(c).alias(c) for c in peliculas.columns]).show())
print('Totales')
print(peliculas.select([F.count(c).alias(c) for c in peliculas.columns]).show())

Distintos
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+
|title_id|primary_title|original_title|premiered|runtime_minutes|titulo_espanol|genres|rating|votes|
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+
|    4937|         4824|          4835|       98|            147|          3876|   368|    75| 4833|
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+

None
Totales
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+
|title_id|primary_title|original_title|premiered|runtime_minutes|titulo_espanol|genres|rating|votes|
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+
|    5050|         5050|          5050|     5050|           5050|          5050|  5050|  5050| 5050|
+--------+-------------+--------------+---------+---------------+--

In [ ]:
peliculas= peliculas.drop_duplicates(['title_id'])

In [ ]:
peliculas.count()

4937

In [ ]:
print('Distintos')
print(peliculas.select([F.countDistinct('title_id').alias(c) for c in peliculas.columns]).show())
print('Totales')
print(peliculas.select([F.count('title_id').alias(c) for c in peliculas.columns]).show())

Distintos
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+
|title_id|primary_title|original_title|premiered|runtime_minutes|titulo_espanol|genres|rating|votes|
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+
|    4937|         4937|          4937|     4937|           4937|          4937|  4937|  4937| 4937|
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+

None
Totales
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+
|title_id|primary_title|original_title|premiered|runtime_minutes|titulo_espanol|genres|rating|votes|
+--------+-------------+--------------+---------+---------------+--------------+------+------+-----+
|    4937|         4937|          4937|     4937|           4937|          4937|  4937|  4937| 4937|
+--------+-------------+--------------+---------+---------------+--

In [ ]:
#Ya tenemos nuestro dataset retocado y bonito

##3. Crea la función que nos permite calcular la media de IMDb.

+ Info: https://help.imdb.com/article/imdb/track-movies-tv/weighted-average-ratings/GWT2DSBYVT2F25SK?ref_=helpsect_pro_2_8#  
+ Info: https://help.imdb.com/article/imdb/track-movies-tv/ratings-faq/G67Y87TFYYP6TWAV?ref_=helpms_helpart_inline#  


**Fórmula Weighted Rating de IMDb**

The following formula is used to calculate the Top Rated 250 titles. This formula provides a true 'Bayesian estimate', which takes into account the number of votes each title has received, minimum votes required to be on the list, and the mean vote for all titles:

**$$WR=\frac{Rv+Cm}{v+m}$$**

Where:

**R** = average for the movie (mean) = (rating)

**v** = number of votes for the movie = (votes)

**m** = minimum votes required to be listed in the Top Rated list (currently 25,000)

**C** = the mean vote across the whole report

<font color="Grey">Please be aware that the Top Rated Movies Chart only includes theatrical features: shorts, TV movies, miniseries and documentaries are not included in the Top Rated Movies Chart. The Top Rated TV Shows Chart includes TV Series, but not TV episodes or Movies.




In [ ]:
#m= número mínimo de votos, vamos a poner 250.000
m=250000

In [ ]:
c=peliculas.agg(F.mean('rating')).collect()[0][0]
c

6.7602997771926265

In [ ]:
def WR(R,v, c=c,m=m):
  return round((R*v+C*m)/(v+m),3)

In [ ]:
wr_udf=F.udf(WR, T.DoubleType())

##4. Prepara los datos que quieras mostrar para cada uno de los títulos.

In [ ]:
peliculas.show(truncate=False)

+---------+--------------------------------------------+--------------------------------------------+---------+---------------+-------------------------------------------------+--------------------------+------+------+
|title_id |primary_title                               |original_title                              |premiered|runtime_minutes|titulo_espanol                                   |genres                    |rating|votes |
+---------+--------------------------------------------+--------------------------------------------+---------+---------------+-------------------------------------------------+--------------------------+------+------+
|tt0032553|The Great Dictator                          |The Great Dictator                          |1940     |125            |El gran dictador                                 |Comedy,Drama,War          |8.5   |187390|
|tt0032599|His Girl Friday                             |His Girl Friday                             |1940     |92           

In [ ]:
#Esto sería con pandas!!!
#Lo que voy a hacer no debe hacerse para datasets inmensos, porque meto en rom al pasar a pandas

#peliculas_pandas= peliculas.toPandas()

#Creo una variable que contiene una lista de tuplas con todas las pelis (primary_title) que (en titulo tienen) ¿?, y su indice

#a=[(fila[1],indice_fila) for indice_fila, fila in peliculas_pandas.iterrows() if fila[5] == '¿?']

#Hago un dataframe con la lista de tuplas

#A=pd.DataFrame(a)

#Itero sobre el dataframe peliculas_pandas, de manera que cambie el titulo_espanol que era ¿? por el primary_title

'''for i in A[1]:
      peliculas_pandas.titulo_espanol[i] = peliculas_pandas.primary_title[i]'''

#Guardo el dataset

#peliculas_pandas.to_csv('/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark/peliculas_pandas.csv')

#Y lo leo con Spark

#peliculas_spark= spark.read.csv('/content/drive/My Drive/Colab Notebooks/Ejercicios Estudiar para Test de Nivel Spark/Práctica de Máster Spark/peliculas_pandas.csv',inferSchema=True, header=True, quote='"',sep=',').cache()

/usr/local/lib/python3.6/dist-packages/pyspark/sql/pandas/conversion.py:88: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.15.1 must be installed; however, your version was 0.14.1.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [ ]:
#Ahora voy a hacerlo con Spark, pero sobre el dataset ya tuneado, así me quito meter en memoria más columnas

In [ ]:
#Tuneo
peliculas_df= (
    peliculas
    .withColumnRenamed('primary_title', 'Título Inglés')
    .withColumnRenamed('titulo_espanol', 'Título')
).select('Título Inglés','Título','rating','votes')

In [ ]:
peliculas_df.show(truncate=False)

+--------------------------------------------+-------------------------------------------------+------+------+
|Título Inglés                               |Título                                           |rating|votes |
+--------------------------------------------+-------------------------------------------------+------+------+
|The Great Dictator                          |El gran dictador                                 |8.5   |187390|
|His Girl Friday                             |Luna nueva                                       |7.9   |49723 |
|Diamonds Are Forever                        |Diamantes para la eternidad                      |6.6   |88793 |
|The Goonies                                 |Los Goonies                                      |7.8   |224926|
|Leaving Las Vegas                           |¿?                                               |7.5   |108836|
|Liar Liar                                   |Mentiroso compulsivo                             |6.9   |266074|
|

In [ ]:
#Filtro para quedarme con un dataeset sólo con los ¿?
desconocido=peliculas_df.filter(F.col('Título')=='¿?')
desconocido.limit(5).show()

+--------------------+------+------+------+
|       Título Inglés|Título|rating| votes|
+--------------------+------+------+------+
|   Leaving Las Vegas|    ¿?|   7.5|108836|
|       Secret Window|    ¿?|   6.6|173329|
|Synecdoche, New York|    ¿?|   7.6| 75047|
|          The Rocker|    ¿?|   6.2| 35746|
|         A Wednesday|    ¿?|   8.1| 69788|
+--------------------+------+------+------+



In [ ]:
#Ahora lo que hago es darle a la columna Título, los valores de Títulos Inglés
desconocido=desconocido.withColumn('Título', F.col('Título Inglés'))
desconocido.show()

+--------------------+--------------------+------+------+
|       Título Inglés|              Título|rating| votes|
+--------------------+--------------------+------+------+
|   Leaving Las Vegas|   Leaving Las Vegas|   7.5|108836|
|       Secret Window|       Secret Window|   6.6|173329|
|Synecdoche, New York|Synecdoche, New York|   7.6| 75047|
|          The Rocker|          The Rocker|   6.2| 35746|
|         A Wednesday|         A Wednesday|   8.1| 69788|
|   Our Idiot Brother|   Our Idiot Brother|   6.4| 81725|
| The Meaning of Life| The Meaning of Life|   7.6|107459|
|          Roger & Me|          Roger & Me|   7.4| 25002|
|      The Full Monty|      The Full Monty|   7.2| 94406|
|     Fahrenheit 9/11|     Fahrenheit 9/11|   7.5|123466|
|     Hannibal Rising|     Hannibal Rising|   6.2| 99365|
|         Bad Grandpa|         Bad Grandpa|   6.5| 86726|
|        Come and See|        Come and See|   8.3| 46146|
|            EuroTrip|            EuroTrip|   6.6|185660|
|     Take Thi

In [ ]:
#Creo la tabla con las conocidas
conocidas=peliculas_df.filter(F.col('Título')!='¿?')
conocidas.limit(5).show()

+--------------------+--------------------+------+------+
|       Título Inglés|              Título|rating| votes|
+--------------------+--------------------+------+------+
|  The Great Dictator|    El gran dictador|   8.5|187390|
|     His Girl Friday|          Luna nueva|   7.9| 49723|
|Diamonds Are Forever|Diamantes para la...|   6.6| 88793|
|         The Goonies|         Los Goonies|   7.8|224926|
|           Liar Liar|Mentiroso compulsivo|   6.9|266074|
+--------------------+--------------------+------+------+



In [ ]:
#Uno la tabla conocidas con la tabla desconocido
peliculas_bonito= conocidas.union(desconocido)
peliculas_bonito.show()

+--------------------+--------------------+------+------+
|       Título Inglés|              Título|rating| votes|
+--------------------+--------------------+------+------+
|  The Great Dictator|    El gran dictador|   8.5|187390|
|     His Girl Friday|          Luna nueva|   7.9| 49723|
|Diamonds Are Forever|Diamantes para la...|   6.6| 88793|
|         The Goonies|         Los Goonies|   7.8|224926|
|           Liar Liar|Mentiroso compulsivo|   6.9|266074|
|Star Wars: Episod...|Star Wars: Episod...|   7.5|658747|
|     Treasure Planet|El planeta del te...|   7.2| 93810|
|         Bring It On|         A por todas|   6.0| 82764|
|              Eragon|              Eragon|   5.1|118507|
|      28 Weeks Later|  28 semanas después|   7.0|246310|
|Take Me Home Tonight| Llévame a casa nena|   6.3| 49417|
|  The Brothers Bloom|  Los hermanos Bloom|   6.8| 47049|
|      The Other Guys|       Los otros dos|   6.6|227120|
|     Horrible Bosses|Cómo acabar con t...|   6.9|405165|
|             

In [ ]:
#Compruebo que esté ok!!
peliculas_bonito.filter(F.col('Título')=='Leaving Las Vegas').show()

+-----------------+-----------------+------+------+
|    Título Inglés|           Título|rating| votes|
+-----------------+-----------------+------+------+
|Leaving Las Vegas|Leaving Las Vegas|   7.5|108836|
+-----------------+-----------------+------+------+



In [ ]:
#Por último, voy a probar que peliculas_df y peliculas_bonito tengan el mismo número de valores
if peliculas_df.count()==peliculas_bonito.count():
  print (True)
else:
  print (False)

True


In [ ]:
#Ahora que tenemos todos los títulos en español, me quedo sólo con esa columna
pelis_df=peliculas_bonito.select('Título','rating','votes')
pelis_df.show()

+-------------------------------------------------+------+------+
|Título                                           |rating|votes |
+-------------------------------------------------+------+------+
|El gran dictador                                 |8.5   |187390|
|Luna nueva                                       |7.9   |49723 |
|Diamantes para la eternidad                      |6.6   |88793 |
|Los Goonies                                      |7.8   |224926|
|Mentiroso compulsivo                             |6.9   |266074|
|Star Wars: Episodio III - La venganza de los Sith|7.5   |658747|
|El planeta del tesoro                            |7.2   |93810 |
|A por todas                                      |6.0   |82764 |
|Eragon                                           |5.1   |118507|
|28 semanas después                               |7.0   |246310|
|Llévame a casa nena                              |6.3   |49417 |
|Los hermanos Bloom                               |6.8   |47049 |
|Los otros

In [ ]:
#Filtramos por películas de más de 250.000 votos
pelis_df=pelis_df.filter(F.col('votes')>250.000)


In [ ]:
#Añadimos la columna que se genera con la udf que hemos creado y la dejo bonita
peliculas_score=pelis_df.withColumn("Score IMDB", wr_udf("rating","votes")).select('Título', 'Score IMDB')
                 

In [ ]:
peliculas_score.show()

+--------------------+----------+
|              Título|Score IMDB|
+--------------------+----------+
|    El gran dictador|     7.506|
|          Luna nueva|     6.949|
|Diamantes para la...|     6.718|
|         Los Goonies|     7.253|
|Mentiroso compulsivo|     6.832|
|Star Wars: Episod...|     7.297|
|El planeta del te...|      6.88|
|         A por todas|     6.571|
|              Eragon|     6.226|
|  28 semanas después|     6.879|
| Llévame a casa nena|     6.684|
|  Los hermanos Bloom|     6.767|
|       Los otros dos|     6.684|
|Cómo acabar con t...|     6.847|
|Puedes confiar en mí|      6.78|
|Kingsman: Servici...|     7.409|
|    Esperando al rey|     6.674|
|Nerve: Un juego s...|     6.683|
|     El hijo de Saúl|     6.864|
|Dos padres por de...|     6.627|
+--------------------+----------+
only showing top 20 rows



##5. Extrae el top 25 del ranking. Se valora la creatividad.

In [ ]:
#Ordenamos por las de mayor Score y filtramos las 25 primeras
peliculas_score.orderBy(F.desc('Score IMDB')).limit(25).toPandas()

/usr/local/lib/python3.6/dist-packages/pyspark/sql/pandas/conversion.py:88: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.15.1 must be installed; however, your version was 0.14.1.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,Título,Score IMDB
0,Cadena perpetua,9.037
1,El padrino,8.849
2,El caballero oscuro,8.766
3,Pulp Fiction,8.626
4,El señor de los anillos: El retorno del rey,8.601
5,El padrino: Parte II,8.565
6,Origen,8.563
7,El club de la lucha,8.543
8,Forrest Gump,8.534
9,El señor de los anillos: La comunidad del anillo,8.517


In [ ]:
#Hacemos el Ranking
pelis_rank= peliculas_score.withColumn('Rank', F.rank().over(Window.orderBy(F.desc('Score IMDB')))).select('Rank','Título','Score IMDB').limit(25).toPandas()

/usr/local/lib/python3.6/dist-packages/pyspark/sql/pandas/conversion.py:88: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.15.1 must be installed; however, your version was 0.14.1.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [ ]:
pelis_rank

,Rank,Título,Score IMDB
0,1,Cadena perpetua,9.037
1,2,El padrino,8.849
2,3,El caballero oscuro,8.766
3,4,Pulp Fiction,8.626
4,5,El señor de los anillos: El retorno del rey,8.601
5,6,El padrino: Parte II,8.565
6,7,Origen,8.563
7,8,El club de la lucha,8.543
8,9,Forrest Gump,8.534
9,10,El señor de los anillos: La comunidad del anillo,8.517
